# Using CodeLlama and [CodeAlpaca_20K](https://huggingface.co/datasets/HuggingFaceH4/CodeAlpaca_20K) finetune for task code generation instructions.

In [ ]:
!pip install datasets==2.14.1

In [2]:
#from datasets import load_dataset
#import pandas as pd

# Load the dataset
#dataset = load_dataset("hrangel/newsql")

In [3]:
#!pip install datasets -q

In [4]:
!pip install transformers evaluate datasets accelerate -q


In [5]:
import pandas as pd
data = pd.read_csv('/kaggle/input/ketodiet/sql3.csv')

In [6]:
from datasets import list_datasets, load_dataset, DatasetInfo
from huggingface_hub import list_datasets, dataset_info

In [7]:
#from datasets import load_dataset
#dataset = load_dataset("hrangel/datasql")

# Add your WanDB API Key
To add API key, go to Add-ons -> Secrets  

In [8]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
personal_key_for_api = user_secrets.get_secret("mysecret")

! wandb login $personal_key_for_api

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
Traceback (most recent call last):
  File "/opt/conda/bin/wandb", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/opt/conda/lib/python3.10/sit

# Install packages


In [9]:
!pip install -q -U accelerate==0.23.0 peft==0.5.0 bitsandbytes==0.41.1 transformers==4.34 trl==0.7.2

# Imports

In [10]:
import torch
from transformers import (
    AutoModelForCausalLM, 
    CodeLlamaTokenizer,
    default_data_collator, 
    Trainer, 
    TrainingArguments,
    TrainerCallback,
    BitsAndBytesConfig,
    AutoTokenizer,
)
from contextlib import nullcontext
from tqdm import tqdm
import json
import copy
import datasets
from peft import LoraConfig, PeftConfig
from transformers import default_data_collator, Trainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Create tokenizer function

# Load model and tokenizer

In [11]:
model_name = "codellama/CodeLlama-7b-Instruct-hf"

# dataset_id = "HuggingFaceH4/CodeAlpaca_20K"

# compute_dtype = getattr(torch, "float16")

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtyp=torch.bfloat16
# )

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    trust_remote_code=True,
    load_in_8bit=True
#     quantization_config=bnb_config, 
)

model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True,
                                         )
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Prepare training data

In [12]:
data.iloc[1]

Question               Dame la hora y el profesor del grupo A
SQL         SELECT hora, profesor FROM escuela WHERE GRUPO...
Answer           El grupo A1 tiene las clases y el profesor :
Context     CREATE TABLE escuela (\n    CUATRIMESTRE INT,\...
Name: 1, dtype: object

In [13]:
data[:1]

,Question,SQL,Answer,Context
0,Dame todas las materias impartidas por el prof...,SELECT MATERIA FROM escuela WHERE PROFESOR = '...,José Roderick Mireles Moreno imparte las mate...,"CREATE TABLE escuela (\n CUATRIMESTRE INT,\..."


In [14]:
instruction_list = []
for i in range(len(data)):
    sample = data.iloc[i]
    instruction = f"""{sample['Context']}
    
-- Using valid SQLite, answer the following questions for the tables provided above.

-- {sample['Answer']}

{sample['SQL']}
"""
    if len(instruction) < 1024:
        instruction_list.append(instruction)

In [15]:
instruction_list[0]

"CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- José Roderick Mireles Moreno imparte las  materias de\n\nSELECT MATERIA FROM escuela WHERE PROFESOR = 'JOSE RODERICK MIRELES\xa0MORENO'\n"

In [16]:
"""def count_create_table(instruction):
    
    cnt = 0
    words = instruction.split()
    for i in range(len(words) - 1):
        if words[i] == 'CREATE' and words[i+1] == 'TABLE':
            cnt += 1
           
    return cnt"""

"def count_create_table(instruction):\n    \n    cnt = 0\n    words = instruction.split()\n    for i in range(len(words) - 1):\n        if words[i] == 'CREATE' and words[i+1] == 'TABLE':\n            cnt += 1\n           \n    return cnt"

In [17]:
#cnt_list = []
#for instruct in instruction_list:
  #  if count_create_table(instruct) > 1:
  #      cnt_list.append(count_create_table(instruct))

In [18]:
import pandas as pd
train_df = pd.DataFrame(list(zip(instruction_list[:60])), columns=['text'])

valid_df = pd.DataFrame(list(zip(instruction_list[61:])), columns=['text'])

In [19]:
from datasets import Dataset
train = Dataset.from_pandas(train_df)
valid = Dataset.from_pandas(valid_df)

In [20]:
def tokenize(data_point):
    result = tokenizer(
        data_point['text'],
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

In [21]:
train_tokenized = train.map(tokenize)
valid_tokenized = valid.map(tokenize)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

# Set up training arguments and LoRA

In [22]:
from transformers import DataCollatorForSeq2Seq

In [23]:
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=4,
        lora_alpha=64,
        lora_dropout=0.1,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config


model, lora_config = create_peft_config(model)

training_arguments = TrainingArguments(
    output_dir="logs",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2, # 4
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=100,
    learning_rate=2e-4,
    fp16=True,
    bf16=False,
    group_by_length=True,
    logging_strategy="steps",
    evaluation_strategy='steps',
    eval_steps=100,
    save_strategy="no",
    gradient_checkpointing=False,
)

trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_tokenized,
    eval_dataset=valid_tokenized,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    )
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:122: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


trainable params: 2,097,152 || all params: 6,740,643,840 || trainable%: 0.031112042851977773


In [24]:
import torch; print(torch.__version__)

2.0.0


# Train and save model

In [ ]:
# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained("trained-model")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

# Use model to generate output

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
trainer.push_to_hub()

In [ ]:
tokenizer.push_to_hub("hrangel/mysql")

In [ ]:
print(instruction_list[-3])

In [ ]:
eval_prompt = instruction_list[-2].split("SELECT")[0] + "SELECT"

In [ ]:
eval_prompt

In [1]:
evalua='CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- ¿Cuántos grupos hay en total?\n'
evalua

'CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- ¿Cuántos grupos hay en total?\n'

In [2]:
evalua='CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- '
evalua

'CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- '

In [ ]:
instruccion = input("Dime instruccion: ")

In [6]:
evalua = evalua + instruccion
evalua

'CREATE TABLE escuela (\n    CUATRIMESTRE INT,\n    TURNO VARCHAR(255),\n    IDCARRERAGRUPO VARCHAR(255),\n    CARRERA VARCHAR(255),\n    NO INT,\n    PROFESOR VARCHAR(255),\n    MATERIA VARCHAR(255),\n    escuela VARCHAR(255),\n    DIA VARCHAR(255),\n    CANTIDAD  INT,\n    GRUPO VARCHAR(255),\n    GRUPO_EN_SISTEMA VARCHAR(255),\n    AULA VARCHAR(255)\n)\n    \n-- Using valid SQLite, answer the following questions for the tables provided above.\n\n-- ¿Cuántos grupos hay en total?\n¿Cuántos grupos hay en total?'

In [ ]:
model.eval()
model_input = tokenizer(evalua, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = tokenizer.decode(model.generate(**model_input, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

print(output)

In [ ]:
instruction_list[-2]

In [ ]:
eval_prompt = instruction_list[-3].split('SELECT')[0] + 'SELECT'
eval_prompt

In [ ]:
model.eval()

eval_prompt = instruction_list[-3].split('SELECT')[0] + 'SELECT'
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    output = tokenizer.decode(model.generate(**model_input, max_new_tokens=250, pad_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)

print(output)